# Régression logistique

## 1 Régularisation de de Tikhonov

### Question 1.1

#### Gradient de $f_1$

$$
\frac{\partial f_1}{\partial w_0}(w_0, w) = \frac{1}{n}\sum_{i=1}^n\frac{-y_ie^{-y_i(x_i^T+w_0)}}{1 + e^{-y_i(x_i^T+w_0)}}
$$

#### Matrice hessienne de $f_1$

#### Convexité de $f_1$